# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Queenstown,-45.0322,168.6610,14.76,68,97,2.29,NZ,2024-11-06
1,1,Margaret River,-33.9532,115.0769,10.08,74,10,2.79,AU,2024-11-06
2,2,Susuman,62.7805,148.1536,-34.17,100,93,0.30,RU,2024-11-06
3,3,Whitehorse,60.7216,-135.0549,2.83,59,75,0.00,CA,2024-11-06
4,4,Hermanus,-34.4175,19.2361,16.02,77,2,7.23,ZA,2024-11-06


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.65,
    frame_height = 600,
    frame_width = 800,
    title = "Cities By Random Selection, Sized By Humidity"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = {
    'Max Temp': [20, 30],
    'Humidity': [0, 60],
    'Cloudiness': [0, 20],
    'Wind Speed': [0, 3]
}
low = 0
high = 1

filter = ((city_data_df['Max Temp'] >= criteria['Max Temp'][low]) & (city_data_df['Max Temp'] <= criteria['Max Temp'][high]) &
          (city_data_df['Humidity'] >= criteria['Humidity'][low]) & (city_data_df['Humidity'] <= criteria['Humidity'][high]) &
          (city_data_df['Cloudiness'] >= criteria['Cloudiness'][low]) & (city_data_df['Cloudiness'] <= criteria['Cloudiness'][high]) &
          (city_data_df['Wind Speed'] >= criteria['Wind Speed'][low]) & (city_data_df['Wind Speed'] <= criteria['Wind Speed'][high]))

filtered_df = city_data_df[filter]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
112,112,Koné,-21.0608,164.8619,26.04,54,7,1.68,NC,2024-11-06
118,118,Fort Bragg,35.1403,-79.1541,28.81,60,0,2.06,US,2024-11-06
134,134,Tazacorte,28.6429,-17.9326,20.49,51,20,2.06,ES,2024-11-06
143,143,Antri,20.6665,76.1787,21.43,56,7,1.34,IN,2024-11-06
218,218,Moranbah,-22.0030,148.0433,23.22,50,1,2.23,AU,2024-11-06
239,239,Kintampo,8.0537,-1.7289,24.32,48,0,1.46,GH,2024-11-06
245,245,La Foa,-21.7097,165.8259,24.25,57,0,1.07,NC,2024-11-06
434,434,Ola,35.0323,-93.2232,23.27,59,0,2.09,US,2024-11-06
436,436,Mutuali,-14.8400,37.0042,25.09,51,18,1.43,MZ,2024-11-06
495,495,Essaouira,31.5118,-9.7621,21.75,35,2,1.54,MA,2024-11-06


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
112,Koné,NC,-21.0608,164.8619,54,
118,Fort Bragg,US,35.1403,-79.1541,60,
134,Tazacorte,ES,28.6429,-17.9326,51,
143,Antri,IN,20.6665,76.1787,56,
218,Moranbah,AU,-22.0030,148.0433,50,
239,Kintampo,GH,8.0537,-1.7289,48,
245,La Foa,NC,-21.7097,165.8259,57,
434,Ola,US,35.0323,-93.2232,59,
436,Mutuali,MZ,-14.8400,37.0042,51,
495,Essaouira,MA,31.5118,-9.7621,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address_response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address_response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Koné - nearest hotel: L'escale
Fort Bragg - nearest hotel: No hotel found
Tazacorte - nearest hotel: Apartamentos Atlantis
Antri - nearest hotel: No hotel found
Moranbah - nearest hotel: No hotel found
Kintampo - nearest hotel: No hotel found
La Foa - nearest hotel: Oasis de Tendea
Ola - nearest hotel: No hotel found
Mutuali - nearest hotel: No hotel found
Essaouira - nearest hotel: Tweets Hotel
Makoua - nearest hotel: No hotel found
Southern Pines - nearest hotel: Jefferson Inn
Wau - nearest hotel: Safari Park Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
112,Koné,NC,-21.0608,164.8619,54,L'escale
118,Fort Bragg,US,35.1403,-79.1541,60,No hotel found
134,Tazacorte,ES,28.6429,-17.9326,51,Apartamentos Atlantis
143,Antri,IN,20.6665,76.1787,56,No hotel found
218,Moranbah,AU,-22.0030,148.0433,50,No hotel found
239,Kintampo,GH,8.0537,-1.7289,48,No hotel found
245,La Foa,NC,-21.7097,165.8259,57,Oasis de Tendea
434,Ola,US,35.0323,-93.2232,59,No hotel found
436,Mutuali,MZ,-14.8400,37.0042,51,No hotel found
495,Essaouira,MA,31.5118,-9.7621,35,Tweets Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.65,
    frame_height = 600,
    frame_width = 800,title = f"""Hotels Near Cities with Ideal Conditions
(Max Temp {criteria['Max Temp'][0]}-{criteria['Max Temp'][1]}C, Humidity {criteria['Humidity'][0]}-{criteria['Humidity'][1]}%, Cloudiness {criteria['Cloudiness'][0]}-{criteria['Cloudiness'][1]}%, Wind Speed {criteria['Wind Speed'][0]}-{criteria['Wind Speed'][1]}m/s)""",
    hover_cols = ["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)